In [4]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np
import warnings

!pip install lifelines
!pip install openpyxl
!pip install xlrd

from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation
from lifelines.utils import concordance_index

    100% |████████████████████████████████| 112kB 2.1MB/s 


In [2]:
#Grab Agency Names

agency_query = bq.Query('SELECT agency_cat \
                        FROM `w210-fedwork.processed_data.status_dynamic_v3` \
                        GROUP BY agency_cat')
df = agency_query.execute(output_options=bq.QueryOutput.dataframe()).result()

agencies = list(df['agency_cat'])

agencies_clean = []

small_agencies = ['ZU','ZR','ZS','ZP','ZL','YF','YE','UT','UJ','TS','RZ','RS','RF','RE','QQ','OS','NS','NP','NM',
                  'NK','MA','IF','HW','HT','HP','HB','GY','GX','GQ','GO','GM','GG','GE','FK','FJ','FI','EW',
                  'EQ','EO','DQ','DF','DB','CX','CF','CE','CC','BZ','BW','BT','BK','AW','AP','AN','YB','JL','CG',
                 'BH','EX','IW','DG','ZN','RO','YD','ZW','IG','VD','YG','EM','EB','GI','YA','ZT','VB','PC','DA',
                 'GN','FR','ZZ','ZQ','AI']

medium_agencies = ['AH','AU','BD','BF','BG','BO','CT','EB','EC','FL','FM','FW','GB','GJ','GW','HD','HF','KS',
                       'LF','LT','MC','MI','PU','RH','RR','SK','SS','TB','TC','TN']

already_complete = ['NU','FW','TD','CM','TR','FQ','KS','AH','MI','BD','GB','GJ','AF','AU','ST','FM','EE','HF','DD',
                   'GS','NL','OM','BO','HD','NV','NN','AM','PU','FT','VA','DL','HS','SZ','TN','SB','EC','CT','DN',
                    'LP','SK','RR','FC','FD','ED','HU','FL','FY','AA','IB','BG','LT','TB','SS','LF','MC','AR','EP',
                    'BF','AB','SE','TC','DJ','AG','IN','GW','HE','NQ','CU','RH','SM','NF']

to_remove = small_agencies + already_complete

for agency in agencies:
  if agency not in to_remove:
    agencies_clean.append(agency)
    
print agencies_clean
print len(agencies_clean)

[]
0


In [8]:
#Create Model Stats File
!touch model_stats.txt
!echo "agency,n,events,concordence" >model_stats.txt

In [9]:
#Loop to Run Model
warnings.filterwarnings('ignore',category=RuntimeWarning)

for agency in agencies_clean:
  print agency
  query = 'SELECT emp_id, age, basic_pay, los,occupation_fam, superv_status, count(*) as duration, max(attrit) as attrition \
                        FROM `w210-fedwork.processed_data.status_dynamic_v3` \
                        WHERE year > 2003 AND agency_cat = "'+agency+'" AND (occupation_cat = "P" OR occupation_cat = "T") AND basic_pay>0 AND nsftp_ind = "1" \
                        GROUP BY emp_id,los,age, basic_pay,occupation_fam,superv_status'
  data_for_model = bq.Query(query)
  df = data_for_model.execute(output_options=bq.QueryOutput.dataframe()).result()
  
  df['attrition'] = df['attrition'].fillna(0)
  df_no_nan = df.drop(['emp_id'],1)
  
  df_with_dummies = pd.get_dummies(df_no_nan)
  df_with_dummies['basic_pay'] = df_with_dummies['basic_pay']/1000
  df_with_dummies['basic_pay_squared'] = (df_with_dummies['basic_pay'])**2
  
  age_to_drop = str(df_with_dummies.filter(like='age').columns[0])
  los_to_drop = str(df_with_dummies.filter(like='los').columns[0])
  occupation_to_drop = str(df_with_dummies.filter(like='occupation_fam').columns[0])
  superv_status_to_drop = str(df_with_dummies.filter(like='superv_status').columns[0])
  
  final_df=df_with_dummies.drop([age_to_drop,los_to_drop,occupation_to_drop,superv_status_to_drop],1)
  
  cph = CoxPHFitter(penalizer=0.1)
  cph.fit(final_df, duration_col='duration', event_col='attrition')
  concord_index = concordance_index(cph.durations,-cph.predict_partial_hazard(cph.data).values.ravel(),cph.event_observed)
  
  with open('model_stats.txt','a') as text_file:
    text_file.write('\n{},{},{},{}'.format(agency,final_df.shape[0],np.sum(cph.event_observed),concord_index))
  
  file_title = str(agency)+'_results.xlsx'
  
  writer = pd.ExcelWriter(file_title)
  cph.summary.transpose().to_excel(writer,'model_results')
  cph.baseline_hazard_.transpose().to_excel(writer,'baseline_hazard')
  cph.baseline_cumulative_hazard_.transpose().to_excel(writer,'baseline_cumulative_hazard')
  writer.save()

NU
FW
SE
TC
AR
EP
BF
AB
DD
GS
SM
NF
DL
VA
HS
SZ
SB
TN
EC
CT
ST
FM
EE
HF
HU
ED
FL
FY
AA
NV
NN
FT
AM
PU
IB
TB
BG
LF
SS
LT
MC
TD
TR
FQ
CM
AH
KS
GB
BD
GJ
MI
AF
AU
DJ
IN
AG
GW
HE
RH
NQ
CU
LP
DN
FD
SK
FC
RR
OM
HD
NL
BO


In [22]:
#Loop to Combine Results

model_results_combined = pd.DataFrame()
baseline_hazard_combined = pd.DataFrame()
baseline_cumulative_hazard_combined = pd.DataFrame()

for agency in already_complete:
  file_title = str(agency)+'_results.xlsx'
  
  results_input = pd.read_excel(file_title,'model_results', index_col = 0)
  results_input['agency'] = agency
  model_results_combined = model_results_combined.append(results_input, ignore_index = False)
  
  baseline_hazard_input = pd.read_excel(file_title,'baseline_hazard', index_col = 0)
  baseline_hazard_input['agency'] = agency
  baseline_hazard_combined = baseline_hazard_combined.append(baseline_hazard_input, ignore_index = True)
  
  baseline_cumulative_hazard_input = pd.read_excel(file_title,'baseline_cumulative_hazard', index_col = 0)
  baseline_cumulative_hazard_input['agency'] = agency
  baseline_cumulative_hazard_combined = baseline_cumulative_hazard_combined.append(baseline_cumulative_hazard_input, ignore_index = True)
  
writer = pd.ExcelWriter('combined_model_results.xlsx')
model_results_combined.to_excel(writer,'model_results')
baseline_hazard_combined.to_excel(writer,'baseline_hazard')
baseline_cumulative_hazard_combined.to_excel(writer,'baseline_cumulative_hazard')
writer.save()